In [ ]:
import matplotlib.pyplot as plt
import glob
import numpy as np
import re
import cmcrameri.cm as cmc
import matplotlib.colors as mcolors

# Create a shifted version of the Oslo colormap by sampling from 0 to 0.90
cmap_original = cmc.oslo
shifted_colors = cmap_original(np.linspace(0, 0.90, 256))
shifted_oslo = mcolors.LinearSegmentedColormap.from_list("shifted_oslo", shifted_colors)

# Get all saved files in the directory
file_paths = sorted(glob.glob("/Users/braydennoh/Documents/phasespace/2.18/shift/t_d_valid_freq_*.txt"))

# Define normalization with LogNorm for correct log scaling
norm = mcolors.LogNorm(vmin=1e-20, vmax=1e-1)

# Create figure and axis
fig, ax = plt.subplots(figsize=(4, 3))

# Loop through each file, extract the perturbation value, and plot
for file_path in file_paths:
    match = re.search(r"t_d_valid_freq_([\d\.Ee+-]+)\.txt", file_path)
    if match:
        perturbation_value = float(match.group(1))
        data = np.loadtxt(file_path)

        # If data is 1D and has 2 elements, reshape it
        if data.ndim == 1 and data.size == 2:
            data = data.reshape(1, 2)
        if data.ndim < 2 or data.shape[1] < 2:
            continue

        color = shifted_oslo(norm(perturbation_value))
        ax.plot(data[:, 0], np.log(data[:, 1]), color=color, lw=1, label=f"{perturbation_value:.1e}")

# Create a manual colorbar with logarithmically spaced ticks
cbar_ticks = [1e-20, 1e-15, 1e-10, 1e-5, 1e-1]
sm = plt.cm.ScalarMappable(cmap=shifted_oslo, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, ticks=cbar_ticks)
cbar.set_label("Perturbation Magnitude")
cbar.ax.set_yticklabels([f"{tick:.0e}" for tick in cbar_ticks])  # Format ticks in scientific notation

ax.set_xlabel('Time')
ax.set_ylabel('log(Separation)')

ax.set_xlim([0, 1000])

plt.tight_layout()
plt.show()
